# Running RBAtools for RBA variability analysis

## Import packages

In [1]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import rba
import pickle

from ipywidgets import IntProgress
from IPython.display import display

from rbatools.rba_session import SessionRBA

## Define functions

In [2]:
# function to record only protein results
# based on SessionRBA method record_results
import rbatools._auxiliary_functions as _auxiliary_functions

def rec_prots(sessionRBA, run_name: str):
    if not hasattr(sessionRBA, 'Results'):
            sessionRBA.Results = {'Reactions': pd.DataFrame(index=list(sessionRBA.ModelStructure.ReactionInfo.Elements.keys())),
                            'Enzymes': pd.DataFrame(index=list(sessionRBA.ModelStructure.EnzymeInfo.Elements.keys())),
                            'Processes': pd.DataFrame(index=[sessionRBA.ModelStructure.ProcessInfo.Elements[i]['ID']+'_machinery' for i in sessionRBA.ModelStructure.ProcessInfo.Elements.keys()]),
                            'Proteins': pd.DataFrame(index=list(sessionRBA.ModelStructure.ProteinMatrix['Proteins'])),
                            'ProtoProteins': pd.DataFrame(index=list(sessionRBA.ModelStructure.ProteinGeneMatrix['ProtoProteins'])),
                            'Constraints': pd.DataFrame(index=sessionRBA.Problem.LP.row_names),
                            'SolutionType': pd.DataFrame(index=['SolutionType']),
                            'ObjectiveFunction': pd.DataFrame(index=sessionRBA.Problem.LP.col_names),
                            'Mu': pd.DataFrame(index=['Mu']),
                            'ObjectiveValue': pd.DataFrame(index=['ObjectiveValue']),
                            'ExchangeFluxes': pd.DataFrame(index=list(sessionRBA.ExchangeMap.keys()))}
    
    sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)

## load the model

In [16]:
xml_dir = 'model\\'
output_dir = 'RBVA_MJahn_model\\'
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

simulation = SessionRBA(xml_dir, lp_solver = 'cplex')

C:\anaconda3\envs\rba-38-env\lib\site-packages\rbatools\rba_problem.py:116: UserWarning: Inconsistent RBA-problem at growth rate = 0
  warnings.warn('Inconsistent RBA-problem at growth rate = 0')


In [17]:
substrate = pd.read_csv('simulation\\substrate_input.csv')
# print(substrate)
mediumDict = simulation.get_medium()

# select medium composition for analysis
mediumRow = 2 # 0 to 2
sessName = '240115_RBVA_mu_0_09'
if mediumRow == 0:
    sessName = sessName + '_80mMFA_fullN'
elif mediumRow == 1:
    sessName = sessName + '_80mMFA_limN'
elif mediumRow == 2:
    sessName = sessName + '_150mMFA_limN'

In [18]:
# print(mediumDict)
print(float(substrate['carbon_conc'].loc[mediumRow]))
print(float(substrate['nitrogen_conc'].loc[mediumRow]))

150.0
3.0


### set medium

In [19]:
carbonConc = float(substrate['carbon_conc'].loc[mediumRow])
nitrogenConc = float(substrate['nitrogen_conc'].loc[mediumRow])

simulation.set_medium({'M_for': carbonConc})
simulation.set_medium({'M_nh4': nitrogenConc})
print(mediumDict)

{'M_cl': 0.131391588434, 'M_cobalt2': 0.000798554197, 'M_fe2': 0.007544112751, 'M_fe3': 0.071939858278, 'M_fru': 0.0, 'M_h': 10.0, 'M_h2o': 10.0, 'M_k': 3.674039238739, 'M_mg2': 0.811424862058, 'M_mobd': 0.000148791073, 'M_na1': 37.974274239318, 'M_nh4': 3.0, 'M_o2': 10.0, 'M_pi': 22.661027567326, 'M_so4': 8.451095914524, 'M_spmd': 0.0, 'M_12ppd__S': 0.0, 'M_15dap': 0.0, 'M_1btol': 0.0, 'M_26dap__M': 0.0, 'M_2ddglcn': 0.0, 'M_2hba': 0.0, 'M_2obut': 0.0, 'M_2pg': 0.0, 'M_34dhbz': 0.0, 'M_3hbz': 0.0, 'M_3hcinnm': 0.0, 'M_3hpppn': 0.0, 'M_3pg': 0.0, 'M_4abut': 0.0, 'M_4flrbz': 0.0, 'M_4flthr': 0.0, 'M_4hbt': 0.0, 'M_4hbz': 0.0, 'M_4hphac': 0.0, 'M_6chq': 0.0, 'M_ac': 0.0, 'M_acac': 0.0, 'M_acald': 0.0, 'M_acgam': 0.0, 'M_acmana': 0.0, 'M_acnam': 0.0, 'M_acon': 0.0, 'M_ade': 0.0, 'M_adip': 0.0, 'M_adn': 0.0, 'M_ala__D': 0.0, 'M_ala__L': 0.0, 'M_alltn': 0.0, 'M_anth': 0.0, 'M_aobzac': 0.0, 'M_arab__L': 0.0, 'M_arg__L': 0.0, 'M_ascb__L': 0.0, 'M_asn__L': 0.0, 'M_aso3': 0.0, 'M_asp__L': 0.0, 

### get reactions and mumax

In [20]:
# get list of all proteins to later relate IDs to proteomics data

# proteinList = simulation.get_proteins()
# proteinInformation = {}
# for prot in proteinList:
#     protInformation = simulation.get_protein_information(protein = prot)
#     proteinInformation[prot] = protInformation

# # export this dictionary to later map reaction names to proper identifiers
# saveFileName = 'C_necator_MJahn_RBA_model_proteins'
# curFolder = os.getcwd()
# savePath = os.path.join(curFolder, saveFileName + '.pkl')

# with open(savePath, 'wb+') as f:
#     pickle.dump(proteinInformation, f)

In [21]:
reactionList = simulation.get_reactions()
# Feasible_Ranges = {}
# for rxn in reactionList:
#     Feasible_Ranges[rxn] = {}
#     Feasible_Ranges[rxn]["Minima"] = []
#     Feasible_Ranges[rxn]["Maxima"] = []

mumax = simulation.find_max_growth_rate()

## Record simple RBA result

In [34]:
print(mumax)
simulation.record_results(run_name="mu_max")
simulation.write_results(session_name = sessName + '_muMax')

0.091796875


In [23]:
simulation.SimulationData.export_csv(output_directory = output_dir + 'manual_RBVA_results\\')

## Simulate variability of each reaction

In [ ]:
# # PROBLEM: only delivers reaction rates, not enzyme concentrations
# simulation.set_growth_rate(Mu = mumax) #set growth-rate to specified value
# # determine feasible ranges of variables in question:
# Feasible_range = simulation.get_feasible_range(variables = reactionList)
# for rxn in reactionList:
#     Feasible_Ranges[rxn]["Minima"].append(Feasible_range[rxn]['Min'])
#     Feasible_Ranges[rxn]["Maxima"].append(Feasible_range[rxn]['Max'])

In [ ]:
# # write my own script to maybe get the k_app values or the protein data directely
# simulation.set_growth_rate(Mu = mumax) #set growth-rate to specified value
# # test with reaction that has non-zero minimum and maximum
# rxn = 'R_DATA3'
# simulation.Problem.clear_objective()
# simulation.Problem.set_objective(inputDict = {rxn: 1.0})
# simulation.Problem.solve_lp(feasible_stati=["optimal","feasible","feasible_only_before_unscaling"],try_unscaling_if_sol_status_is_feasible_only_before_unscaling=False)
# simulation.record_results(run_name = rxn + '_max_result')
# simulation.Problem.set_objective(inputDict = {rxn: -1.0})
# simulation.Problem.solve_lp(feasible_stati=["optimal","feasible","feasible_only_before_unscaling"],try_unscaling_if_sol_status_is_feasible_only_before_unscaling=False)
# simulation.record_results(run_name = rxn + '_min_result')
# simulation.write_results(session_name = rxn) 
# simulation.SimulationData.export_csv(output_directory = output_dir + 'manual_RBVA_results\\')
# # this stores the results in individual csv files, the "ProteinData.csv" one seems to hold the protein concentrations
# # only has values for ~400, maybe only non-zero?
# # rows have KEGG-ID values, need to perhaps run another UniProt-ID search to map this. But the min and max of each row should be what I am interested in

In [22]:
# now as a loop
# reactionList = reactionList[1000:1050] # to test --> DOES WORK!!
# simulation.set_growth_rate(Mu = mumax) #set growth-rate to specified value
simulation.set_growth_rate(Mu = 0.09)

for rxn in reactionList:
    simulation.Problem.clear_objective()
    simulation.Problem.set_objective(inputDict = {rxn: 1.0})
    simulation.Problem.solve_lp(feasible_stati=["optimal","feasible","feasible_only_before_unscaling"],try_unscaling_if_sol_status_is_feasible_only_before_unscaling=False)
#     simulation.record_results(run_name = rxn + '_min')
    rec_prots(simulation, run_name = rxn + '_min')
# edit the record_results funtion to only record the proteome
    simulation.Problem.set_objective(inputDict = {rxn: -1.0})
    simulation.Problem.solve_lp(feasible_stati=["optimal","feasible","feasible_only_before_unscaling"],try_unscaling_if_sol_status_is_feasible_only_before_unscaling=False)
#     simulation.record_results(run_name = rxn + '_max')
    rec_prots(simulation, run_name = rxn + '_max')

simulation.write_results(session_name = sessName) 
simulation.SimulationData.export_csv(output_directory = output_dir + 'manual_RBVA_results\\')

C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\2

C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\2

C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\2

C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\2

C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\2

...

C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\2

C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\2

C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\2

C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\2

C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\207290152.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)
C:\Users\Manuel Bruch\AppData\Local\Temp\ipykernel_16968\2

In [ ]:
# simulation.write_results(session_name = sessName) 
# simulation.SimulationData.export_csv(output_directory = output_dir + 'manual_RBVA_results\\')

save the feasible ranges as a dictionary

In [ ]:
# saveFileName = '240105_RBVA_ranges_150mM_FA_limN'
# curFolder = os.getcwd()
# savePath = os.path.join(curFolder, 'simulation', 'RBVA_results', saveFileName + '.pkl')

# with open(savePath, 'wb+') as f:
#     pickle.dump(Feasible_Ranges, f)